In [3]:
import distributed, sys, joblib, mhcflurry_cloud, mhcflurry, fancyimpute
import distributed.joblib
import joblib
import os, socket, time

In [4]:
#executor = distributed.Executor('10.20.95.31:8786')
executor = distributed.Executor('104.196.185.187:8786')

def foo(n, power=2):
    print("Calculating")
    return n**power

def something(n):
    #time.sleep(1)
    return socket.gethostname()

In [7]:
train_data = mhcflurry.dataset.Dataset.from_csv("../test/data/bdata.2009.mhci.public.1.txt")
# Generate cross validation folds, with imputation.
#imputer = fancyimpute.MICE(n_imputations=2, n_burn_in=1, n_nearest_columns=25)

imputer=None
folds = mhcflurry_cloud.cross_validation_folds(
    train_data,
    n_folds=3,
    imputer=imputer,
    drop_similar_peptides=True,
    alleles=["HLA-A0201", "HLA-A0202"],
    n_jobs=-1,
    verbose=5,
)
folds


8

In [13]:
# Make some models.
models = mhcflurry_cloud.models_grid(
    activation=["tanh", "relu"],
    layer_sizes=[[4], [16], [32], [128], [16,64]],
    embedding_output_dim=[8, 32, 64],
    dropout_probability=[0, .1, .5],
    n_training_epochs=[250])
print(models[0])
len(models)

{'fraction_negative': 0.2, 'impute': False, 'embedding_output_dim': 8, 'n_training_epochs': 250, 'layer_sizes': [4], 'max_ic50': 50000, 'pretrain_decay': '1 / (1+epoch)**2', 'batch_size': 128, 'dropout_probability': 0, 'activation': 'tanh'}


90

In [14]:
start = time.time()
with joblib.parallel_backend('distributed', scheduler_host='104.196.185.187:8786'):
    df = mhcflurry_cloud.train_across_models_and_folds(folds, models)
print("Trained %d models in %f sec" % (len(df), time.time() - start))
df

Trained 540 models in 1021.548257 sec


,allele,fold_num,model_num,train_size,test_size,train_auc,train_tau,train_f1,test_auc,test_tau,...,model_fraction_negative,model_impute,model_embedding_output_dim,model_n_training_epochs,model_layer_sizes,model_max_ic50,model_pretrain_decay,model_batch_size,model_dropout_probability,model_activation
0,HLA-A0201,0,0,6376,1478,0.959499,0.636762,0.846451,0.952503,0.565436,...,0.2,False,8,250,[4],50000,1 / (1+epoch)**2,128,0.0,tanh
1,HLA-A0201,0,1,6376,1478,0.975554,0.678579,0.882870,0.944438,0.548959,...,0.2,False,8,250,[16],50000,1 / (1+epoch)**2,128,0.0,tanh
2,HLA-A0201,0,2,6376,1478,0.986666,0.716709,0.916503,0.929574,0.522471,...,0.2,False,8,250,[32],50000,1 / (1+epoch)**2,128,0.0,tanh
3,HLA-A0201,0,3,6376,1478,0.996195,0.808062,0.945793,0.922403,0.500872,...,0.2,False,8,250,[128],50000,1 / (1+epoch)**2,128,0.0,tanh
4,HLA-A0201,0,4,6376,1478,0.987007,0.718200,0.912203,0.937908,0.523984,...,0.2,False,8,250,"[16, 64]",50000,1 / (1+epoch)**2,128,0.0,tanh
5,HLA-A0201,0,5,6376,1478,0.958234,0.635163,0.840937,0.955114,0.568692,...,0.2,False,8,250,[4],50000,1 / (1+epoch)**2,128,0.0,relu
6,HLA-A0201,0,6,6376,1478,0.973075,0.669501,0.885680,0.942275,0.545694,...,0.2,False,8,250,[16],50000,1 / (1+epoch)**2,128,0.0,relu
7,HLA-A0201,0,7,6376,1478,0.981569,0.701903,0.898921,0.940979,0.553372,...,0.2,False,8,250,[32],50000,1 / (1+epoch)**2,128,0.0,relu
8,HLA-A0201,0,8,6376,1478,0.994031,0.769672,0.938355,0.913153,0.500928,...,0.2,False,8,250,[128],50000,1 / (1+epoch)**2,128,0.0,relu
9,HLA-A0201,0,9,6376,1478,0.982470,0.701695,0.899206,0.938516,0.531171,...,0.2,False,8,250,"[16, 64]",50000,1 / (1+epoch)**2,128,0.0,relu


In [23]:
df["overall_score"] = df.test_auc + df.test_tau + df.test_f1
df.sort_values("overall_score", ascending=False, inplace=True)
df.groupby("allele").head(1).overall_score

10     2.337946
460    2.333442
Name: overall_score, dtype: float64

In [40]:
df.ix[df.allele == allele].overall_score.rank?

Object `overall_score.rank` not found.


In [ ]:
df.ix[df.allele == allele].overall_score.rank

In [ ]:
df.ix[df.allele == allele].overall_score.rank

In [48]:
df["summary_rank"] = -1
for allele in df.allele.unique():
    df.ix[(df.allele == allele), "summary_rank"] = df.ix[df.allele == allele].overall_score.rank(
        method='first', ascending=False).astype(int)
    

In [63]:
df.set_index('allele', drop=False, inplace=True)
list(df.train_f1.iteritems())

[('HLA-A0201', 0.8326349519349272),
 ('HLA-A0202', 0.82798064232292123),
 ('HLA-A0201', 0.84487465872424916),
 ('HLA-A0201', 0.83080745341614903),
 ('HLA-A0201', 0.84187082405345215),
 ('HLA-A0201', 0.85659101883148236),
 ('HLA-A0201', 0.84645098985997103),
 ('HLA-A0202', 0.82494432071269486),
 ('HLA-A0201', 0.84093711467324295),
 ('HLA-A0201', 0.84862385321100908),
 ('HLA-A0202', 0.83519428316212596),
 ('HLA-A0201', 0.85295508274231679),
 ('HLA-A0202', 0.8537020517395183),
 ('HLA-A0202', 0.8210806023029229),
 ('HLA-A0201', 0.84228855721393037),
 ('HLA-A0202', 0.85150143973673387),
 ('HLA-A0201', 0.85357492654260536),
 ('HLA-A0201', 0.8669423497932377),
 ('HLA-A0201', 0.85459499628436963),
 ('HLA-A0201', 0.85228113440197273),
 ('HLA-A0202', 0.84789356984478947),
 ('HLA-A0201', 0.85001199904007674),
 ('HLA-A0201', 0.8472222222222221),
 ('HLA-A0201', 0.89020478657784363),
 ('HLA-A0201', 0.82616352201257859),
 ('HLA-A0201', 0.84279475982532748),
 ('HLA-A0201', 0.8412818571789048),
 ('HLA-

In [79]:
df.ix[df.allele == "HLA-A0201"].groupby("model_num").overall_score.mean().rank(method='first', ascending=False).to_dict()

{0: 6.0,
 1: 29.0,
 2: 45.0,
 3: 79.0,
 4: 42.0,
 5: 3.0,
 6: 27.0,
 7: 34.0,
 8: 55.0,
 9: 36.0,
 10: 1.0,
 11: 9.0,
 12: 17.0,
 13: 28.0,
 14: 8.0,
 15: 2.0,
 16: 7.0,
 17: 22.0,
 18: 40.0,
 19: 15.0,
 20: 86.0,
 21: 71.0,
 22: 78.0,
 23: 30.0,
 24: 82.0,
 25: 84.0,
 26: 72.0,
 27: 63.0,
 28: 46.0,
 29: 74.0,
 30: 21.0,
 31: 61.0,
 32: 70.0,
 33: 88.0,
 34: 51.0,
 35: 14.0,
 36: 41.0,
 37: 69.0,
 38: 83.0,
 39: 50.0,
 40: 4.0,
 41: 19.0,
 42: 31.0,
 43: 49.0,
 44: 16.0,
 45: 13.0,
 46: 24.0,
 47: 38.0,
 48: 75.0,
 49: 26.0,
 50: 73.0,
 51: 68.0,
 52: 60.0,
 53: 20.0,
 54: 77.0,
 55: 87.0,
 56: 66.0,
 57: 58.0,
 58: 59.0,
 59: 56.0,
 60: 23.0,
 61: 67.0,
 62: 76.0,
 63: 89.0,
 64: 62.0,
 65: 11.0,
 66: 43.0,
 67: 64.0,
 68: 85.0,
 69: 47.0,
 70: 10.0,
 71: 18.0,
 72: 35.0,
 73: 48.0,
 74: 32.0,
 75: 5.0,
 76: 33.0,
 77: 37.0,
 78: 80.0,
 79: 25.0,
 80: 81.0,
 81: 53.0,
 82: 54.0,
 83: 12.0,
 84: 65.0,
 85: 90.0,
 86: 52.0,
 87: 39.0,
 88: 57.0,
 89: 44.0}

In [72]:
df.groupby(["allele", "model_num"]).overall_score.mean().rank(method='first', ascending=False)

allele     model_num
HLA-A0201  0              6.0
           1             48.0
           2            104.0
           3            149.0
           4             98.0
           5              3.0
           6             45.0
           7             77.0
           8            116.0
           9             88.0
           10             1.0
           11            10.0
           12            23.0
           13            46.0
           14             8.0
           15             2.0
           16             7.0
           17            32.0
           18            96.0
           19            20.0
           20           170.0
           21           138.0
           22           148.0
           23            50.0
           24           159.0
           25           167.0
           26           139.0
           27           127.0
           28           105.0
           29           143.0
                        ...  
HLA-A0202  60            62.0
           61      

In [51]:
df.ix[df.summary_rank == 1].set_index("allele").model_num.to_dict()

{'HLA-A0201': 10, 'HLA-A0202': 10}

In [27]:
df.groupby("allele").head(1).index

Int64Index([10, 460], dtype='int64')

In [25]:
df.groupby("allele").overall_score.min()

allele
HLA-A0201    1.964580
HLA-A0202    1.929411
Name: overall_score, dtype: float64

In [ ]:
result = executor.map(foo, [5,6,7])


In [ ]:
f = result[2]
f.result()

In [ ]:
f2 = executor.map(something, range(100))
[x.result() for x in f2]

In [ ]:
joblib.Parallel(n_jobs=2)(joblib.delayed(foo)(n, power=2) for n in range(10))

In [ ]:
joblib.Parallel?